RUN THE CODE IN THIS NOTEBOOK HERE: https://colab.research.google.com/drive/1-P9xrSE-l14V0mS0qsHiY_ZmHGxPqx01?usp=sharing

The purpose of the following code is to train a RL model that knows to stay within a certain altitude range in order to maximize its reward.

In [ ]:
#code adapted from: https://github.com/nicknochnack/ReinforcementLearningCourse/blob/main/Project%203%20-%20Custom%20Environment.ipynb

#you need to pip install stable_baselines3 so make sure you run this cell first
!pip install stable_baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 33.6 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=65f1cf8c9369064ddaa0bb39991021b8556398ca4f30497fdc551f225719c0e7
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


# 1. Import Dependencies

In [ ]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 3. Building an Environment

In [ ]:
class FlightEnv(Env):
    """
    This is the environment that we use for reinforcement learning.
    The idea is that the drone will learn to stay within a certain altitude range
    (37 to 39 inclusive) where its reward will be maximized.
    """
    def __init__(self):
        """
        Constructor method
        """
        # Actions we can take (go down, stay at same altitude, or go up)
        self.action_space = Discrete(3)
        # altitude array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start altitude
        self.state = 38 + random.randint(-3,3)
        # Set flight length
        self.flight_length = 60
        
    def step(self, action):
        """
        One step of reinforcement learning. 

        :param action: the action that the drone should take (either 0, 1, or 2),
        0 means go down 1 unit, 1 means stay at the same altitude, and 2 means
        go up 1 unit
        :type action: int
        :return: step information
        :rtype: tuple
        """
        # Apply action
        # 0 -1 = -1 altitude
        # 1 -1 = 0 
        # 2 -1 = 1 altitude
        self.state += action -1 
        # Reduce flight length by 1
        self.flight_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if flight is done
        if self.flight_length <= 0: 
            done = True
        else:
            done = False
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        """
        Renders the environment (we don't use this since we use AirSim as the display)
        """
        # Implement viz
        pass
    
    def reset(self):
        """
        Resets the environment.

        :return: initial state (altitude)
        :rtype: numpy array of floats
        """
        # Reset altitude
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset flight time
        self.flight_length = 60 
        return self.state

In [ ]:
env=FlightEnv()

/usr/local/lib/python3.8/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
env.observation_space.sample()

array([32.38439], dtype=float32)

In [ ]:
env.reset()

array([41.])

# 4. Test Environment

In [ ]:
#this code just tests the environment with random actions

episodes = 5
for episode in range(1, episodes+1):
    #reset the environment and return the initial state (altitude)
    state = env.reset()
    done = False
    #keep track of total reward for the episode
    score = 0 
    while not done:
        env.render()
        #get a random action
        action = env.action_space.sample()
        #use airsim command to do the action
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-46
Episode:2 Score:-58
Episode:3 Score:-34
Episode:4 Score:-36
Episode:5 Score:-58


In [ ]:
env.close()

# 5. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.5    |
| time/              |          |
|    fps             | 352      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -28.5       |
| time/                   |             |
|    fps                  | 409         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004159675 |
|    clip_fraction        | 0.011       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 4.68e-05    |

# 6. Save Model

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(23.6, 54.73426714591143)